# Post training notes / reminders


**ALSO: seem to require force cu121 torch due to changes in unsloth ongoing October 2024 - couldn't reuse same pip installs as I did in september O_o**

---


- this is only 60 steps
- **I think i managed to get the Train on Completions Only stuff to work with the collator (loss does indeed go down and results are sensible with a few test inferences - before I had loss stuck at 0 on previous notebooks attempts)**
- filtered dataset so use only "good samples" see code below (there is metadata for the synthetic dataset)
- **MISTAKE/notebook example:** USES Llama3.1 NOT THE INSTRUCT VERSION as the base model O_o **redo with instruct**

- still can't get the GGUF to work on Kaggle, disk size it seems is problem

In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth


import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Dataset prep

- care large dataset - here i filtered on some kind of quality metric they have on their synthetic data

In [7]:
from datasets import load_dataset


full_pii_ds = load_dataset("gretelai/synthetic_pii_finance_multilingual")

README.md:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/48.4M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/5.42M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50346 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5594 [00:00<?, ? examples/s]

In [8]:
full_pii_ds

DatasetDict({
    train: Dataset({
        features: ['level_0', 'index', 'document_type', 'document_description', 'expanded_type', 'expanded_description', 'language', 'language_description', 'domain', 'generated_text', 'pii_spans', 'conformance_score', 'quality_score', 'toxicity_score', 'bias_score', 'groundedness_score'],
        num_rows: 50346
    })
    test: Dataset({
        features: ['level_0', 'index', 'document_type', 'document_description', 'expanded_type', 'expanded_description', 'language', 'language_description', 'domain', 'generated_text', 'pii_spans', 'conformance_score', 'quality_score', 'toxicity_score', 'bias_score', 'groundedness_score'],
        num_rows: 5594
    })
})

In [9]:
full_pii_ds["train"][0]

{'level_0': 40012,
 'index': 40012,
 'document_type': 'Supply Chain Management Agreement',
 'document_description': 'A legal contract outlining the terms and conditions of managing the flow of goods and services within a supply chain, including responsibilities and performance metrics.',
 'expanded_type': 'Vendor Management Contract',
 'expanded_description': 'This subtype involves the contractual agreements and guidelines for managing vendors within the supply chain. It covers aspects such as vendor selection, performance evaluation, and dispute resolution procedures.',
 'language': 'English',
 'language_description': 'English language as spoken in the United States, the UK, or Canada',
 'domain': 'finance',
 'generated_text': 'SUPPLY CHAIN MANAGEMENT AGREEMENT\n\nThis Supply Chain Management Agreement (the "Agreement") is entered into as of this 1st day of March, 2021 (the "Effective Date"), by and between Cameron-Mcknight, a company organized and existing under the laws of the state

In [10]:
good_data = full_pii_ds.filter(lambda x : x["conformance_score"] > 90)

Filter:   0%|          | 0/50346 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5594 [00:00<?, ? examples/s]

In [11]:
good_data

DatasetDict({
    train: Dataset({
        features: ['level_0', 'index', 'document_type', 'document_description', 'expanded_type', 'expanded_description', 'language', 'language_description', 'domain', 'generated_text', 'pii_spans', 'conformance_score', 'quality_score', 'toxicity_score', 'bias_score', 'groundedness_score'],
        num_rows: 9197
    })
    test: Dataset({
        features: ['level_0', 'index', 'document_type', 'document_description', 'expanded_type', 'expanded_description', 'language', 'language_description', 'domain', 'generated_text', 'pii_spans', 'conformance_score', 'quality_score', 'toxicity_score', 'bias_score', 'groundedness_score'],
        num_rows: 1040
    })
})

In [12]:
import json

In [13]:
# i converted to uppsercase
PII_LABELS = ['ACCOUNT_PIN', 'API_KEY', 'BANK_ROUTING_NUMBER', 'BBAN', 'COMPANY', 'CREDIT_CARD_NUMBER', 'CREDIT_CARD_SECURITY_CODE', 'CUSTOMER_ID', 'DATE', 'DATE_OF_BIRTH', 'DATE_TIME', 'DRIVER_LICENSE_NUMBER', 'EMAIL', 'EMPLOYEE_ID', 'FIRST_NAME', 'IBAN', 'IPV4', 'IPV6', 'LAST_NAME', 'LOCAL_LATLNG', 'NAME', 'PASSPORT_NUMBER', 'PASSWORD', 'PHONE_NUMBER', 'SSN', 'STREET_ADDRESS', 'SWIFT_BIC_CODE', 'TIME', 'USER_NAME']

In [14]:
# ----------- important


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN



# must add to unsloth prompts it says O_o

In [15]:
def process_example(example):
    curr_close_idx = 0
    # iterate over dics of annotations, care account for closing index
    processed_string = []
    
    orig_text = example["generated_text"]
    
    pii_d = json.loads(example["pii_spans"])

    
    for annot_d in pii_d:
        s, e = annot_d["start"], annot_d["end"]
        label = annot_d["label"].upper()
        
        # add string up to this annotation
        if s > curr_close_idx:
            processed_string.append(orig_text[curr_close_idx:s])
        
        # update curr_close_idx
        curr_close_idx = e
        
        # add the <TAG>xxxx</TAG>
        tmp = f"<START OF IDENTIFIABLE INFORMATION : {label}>{orig_text[s:e]}<END OF IDENTIFIABLE INFORMATION : {label}>"
        
        processed_string.append(tmp)
        
    # handle end of sentence
    if curr_close_idx < len(orig_text):
        processed_string.append(orig_text[curr_close_idx:])
    
    pii_sequence_string = ''.join(processed_string)
    
    text_lang = example["language"]
    
    llama3template = f"""Below is a DOCUMENT written in {text_lang} that may contain sensitive identifiable information, such as people's names or telephone numbers.

Find all the sensitive identifiable information in the DOCUMENT and tag these sections of the DOCUMENT.

Use only the tags in the IDENTIFIABLE INFORMATION TAGS list below.

Reply with a LABELLED DOCUMENT which contains all the sensitive identifiable information.

Follow the formatting of the EXAMPLE given below without deviation.

### EXAMPLE:

document: "My name is Bob Smith and you can call me on 0800-134-5813 today."
labelled document: "My name is <START OF IDENTIFIABLE INFORMATION : NAME>Bob Smith<END OF IDENTIFIABLE INFORMATION : NAME> and you can call me on <START OF IDENTIFIABLE INFORMATION : PHONE_NUMBER>0800-134-5813<END OF IDENTIFIABLE INFORMATION : PHONE_NUMBER> today."

### DOCUMENT:

{orig_text}

### IDENTIFIABLE INFORMATION TAGS:

- ACCOUNT_PIN
- API_KEY
- BANK_ROUTING_NUMBER
- BBAN
- COMPANY
- CREDIT_CARD_NUMBER
- CREDIT_CARD_SECURITY_CODE
- CUSTOMER_ID
- DATE
- DATE_OF_BIRTH
- DATE_TIME
- DRIVER_LICENSE_NUMBER
- EMAIL
- EMPLOYEE_ID
- FIRST_NAME
- IBAN
- IPV4
- IPV6
- LAST_NAME
- LOCAL_LATLNG
- NAME
- PASSPORT_NUMBER
- PASSWORD
- PHONE_NUMBER
- SSN
- STREET_ADDRESS
- SWIFT_BIC_CODE
- TIME
- USER_NAME

### LABELLED DOCUMENT:

{pii_sequence_string}"""
    
    return llama3template + EOS_TOKEN # CARE! ADD MANUALLY
    

In [16]:
print(process_example(good_data["train"][0]))

Below is a DOCUMENT written in English that may contain sensitive identifiable information, such as people's names or telephone numbers.

Find all the sensitive identifiable information in the DOCUMENT and tag these sections of the DOCUMENT.

Use only the tags in the IDENTIFIABLE INFORMATION TAGS list below.

Reply with a LABELLED DOCUMENT which contains all the sensitive identifiable information.

Follow the formatting of the EXAMPLE given below without deviation.

### EXAMPLE:

document: "My name is Bob Smith and you can call me on 0800-134-5813 today."
labelled document: "My name is <START OF IDENTIFIABLE INFORMATION : NAME>Bob Smith<END OF IDENTIFIABLE INFORMATION : NAME> and you can call me on <START OF IDENTIFIABLE INFORMATION : PHONE_NUMBER>0800-134-5813<END OF IDENTIFIABLE INFORMATION : PHONE_NUMBER> today."

### DOCUMENT:

SUPPLY CHAIN RESILIENCE FRAMEWORK

This Supply Chain Resilience Framework (the "Agreement") is entered into as of this 1st day of August, 2021, by and betwe

In [18]:
# BUILD TRAIN DATASET

llama3_train_dataset = []

for example in good_data["train"]:
    x = process_example(example)
    if x is not None:
        llama3_train_dataset.append(x)

In [19]:
len(llama3_train_dataset)

9197

In [20]:
import pandas as pd
from datasets import Dataset

# convert to HF dataset
df = pd.DataFrame(llama3_train_dataset)
train_data = Dataset.from_pandas(df.rename(columns={0: "text"}), split="train")

In [26]:
train_data["text"][0]

'Below is a DOCUMENT written in English that may contain sensitive identifiable information, such as people\'s names or telephone numbers.\n\nFind all the sensitive identifiable information in the DOCUMENT and tag these sections of the DOCUMENT.\n\nUse only the tags in the IDENTIFIABLE INFORMATION TAGS list below.\n\nReply with a LABELLED DOCUMENT which contains all the sensitive identifiable information.\n\nFollow the formatting of the EXAMPLE given below without deviation.\n\n### EXAMPLE:\n\ndocument: "My name is Bob Smith and you can call me on 0800-134-5813 today."\nlabelled document: "My name is <START OF IDENTIFIABLE INFORMATION : NAME>Bob Smith<END OF IDENTIFIABLE INFORMATION : NAME> and you can call me on <START OF IDENTIFIABLE INFORMATION : PHONE_NUMBER>0800-134-5813<END OF IDENTIFIABLE INFORMATION : PHONE_NUMBER> today."\n\n### DOCUMENT:\n\nSUPPLY CHAIN RESILIENCE FRAMEWORK\n\nThis Supply Chain Resilience Framework (the "Agreement") is entered into as of this 1st day of Augus

In [27]:
from trl import DataCollatorForCompletionOnlyLM

In [32]:
completions_only_collator = DataCollatorForCompletionOnlyLM(
    instruction_template=tokenizer.eos_token,
    response_template=tokenizer.bos_token,
    tokenizer=tokenizer,
)

In [33]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    data_collator=completions_only_collator, # NOTE -- I ADDED THIS FOR THE COMPLETIONS ONLY STUFF
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 60,

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map (num_proc=2):   0%|          | 0/9197 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [34]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.004 GB of memory reserved.


In [35]:
# go

# TOOD: CHECK LOSS IS NON ZERO SINCE CURRENTLY YTRYING TO GET THE COMPLETIONS ONLY COLLATOR TO WORK DOC ON HF IS UNCLEAr

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,197 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


Step,Training Loss
1,0.826000
2,0.803500
3,0.825200
4,0.798700
5,0.756100
6,0.677300
7,0.697800
8,0.622500
9,0.510100
10,0.459300


In [36]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1865.3669 seconds used for training.
31.09 minutes used for training.
Peak reserved memory = 9.867 GB.
Peak reserved memory for training = 3.863 GB.
Peak reserved memory % of max memory = 66.936 %.
Peak reserved memory for training % of max memory = 26.206 %.


# Inference

In [37]:
inference_prompt_template = """Below is a DOCUMENT written in {text_lang} that may contain sensitive identifiable information, such as people's names or telephone numbers.

Find all the sensitive identifiable information in the DOCUMENT and tag these sections of the DOCUMENT.

Use only the tags in the IDENTIFIABLE INFORMATION TAGS list below.

Reply with a LABELLED DOCUMENT which contains all the sensitive identifiable information.

Follow the formatting of the EXAMPLE given below without deviation.

### EXAMPLE:

document: "My name is Bob Smith and you can call me on 0800-134-5813 today."
labelled document: "My name is <START OF IDENTIFIABLE INFORMATION : NAME>Bob Smith<END OF IDENTIFIABLE INFORMATION : NAME> and you can call me on <START OF IDENTIFIABLE INFORMATION : PHONE_NUMBER>0800-134-5813<END OF IDENTIFIABLE INFORMATION : PHONE_NUMBER> today."

### DOCUMENT:

{orig_text}

### IDENTIFIABLE INFORMATION TAGS:

- ACCOUNT_PIN
- API_KEY
- BANK_ROUTING_NUMBER
- BBAN
- COMPANY
- CREDIT_CARD_NUMBER
- CREDIT_CARD_SECURITY_CODE
- CUSTOMER_ID
- DATE
- DATE_OF_BIRTH
- DATE_TIME
- DRIVER_LICENSE_NUMBER
- EMAIL
- EMPLOYEE_ID
- FIRST_NAME
- IBAN
- IPV4
- IPV6
- LAST_NAME
- LOCAL_LATLNG
- NAME
- PASSPORT_NUMBER
- PASSWORD
- PHONE_NUMBER
- SSN
- STREET_ADDRESS
- SWIFT_BIC_CODE
- TIME
- USER_NAME

### LABELLED DOCUMENT:

"""

In [42]:
query_lang = "English"

query_doc = "Yeah so it's working OK but I need an update sent to Mr. Thomas Kylermanski at 89 Grovewood St. if possible by tomorrow. Login with K9xa123 to get the access if you need it"

In [43]:
FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!

inputs = tokenizer(inference_prompt_template.format(text_lang=query_lang, orig_text=query_doc), return_tensors = "pt").to("cuda")

outputs = model.generate(input_ids = inputs.input_ids,
                         attention_mask = inputs.attention_mask,
                         max_new_tokens = 500)

#tokenizer.batch_decode(outputs)

In [47]:
res = tokenizer.batch_decode(outputs)

# add 0 here - might not work when have several in batch TODO: test with dataset not just 1 sample
start_idx = res[0].index("### LABELLED DOCUMENT:\n\n")

print(start_idx)

generated_response = res[0][start_idx + len("### LABELLED DOCUMENT:\n\n"):]



1450


In [52]:
print(generated_response)

Yeah so it's working OK but I need an update sent to <START OF IDENTIFIABLE INFORMATION : NAME>Mr. Thomas Kylermanski<END OF IDENTIFIABLE INFORMATION : NAME> at <START OF IDENTIFIABLE INFORMATION : STREET_ADDRESS>89 Grovewood St.<END OF IDENTIFIABLE INFORMATION : STREET_ADDRESS> if possible by tomorrow. Login with <START OF IDENTIFIABLE INFORMATION : USER_NAME>K9xa123<END OF IDENTIFIABLE INFORMATION : USER_NAME> to get the access if you need it<|end_of_text|>


In [53]:
query_lang = "French"

query_doc = "oui je vous entends, mais c'est déjà envoyé chez vous au 23 bvd du Centre - je vous invite à me recontacter au 04 78 21 32 01 s'il y a toujours des soucis. On vous enverra un technicien de FibreMax Services pour voir sinon"

In [54]:
inputs = tokenizer(inference_prompt_template.format(text_lang=query_lang, orig_text=query_doc), return_tensors = "pt").to("cuda")

outputs = model.generate(input_ids = inputs.input_ids,
                         attention_mask = inputs.attention_mask,
                         max_new_tokens = 500)

res = tokenizer.batch_decode(outputs)

# add 0 here - might not work when have several in batch TODO: test with dataset not just 1 sample
start_idx = res[0].index("### LABELLED DOCUMENT:\n\n")

print(start_idx)

generated_response = res[0][start_idx + len("### LABELLED DOCUMENT:\n\n"):]

print(generated_response)

1499
oui je vous entends, mais c'est déjà envoyé chez vous au <START OF IDENTIFIABLE INFORMATION : STREET_ADDRESS>23 bvd du Centre<END OF IDENTIFIABLE INFORMATION : STREET_ADDRESS> - je vous invite à me recontacter au <START OF IDENTIFIABLE INFORMATION : PHONE_NUMBER>04 78 21 32 01<END OF IDENTIFIABLE INFORMATION : PHONE_NUMBER> s'il y a toujours des soucis. On vous enverra un technicien de <START OF IDENTIFIABLE INFORMATION : COMPANY>FibreMax Services<END OF IDENTIFIABLE INFORMATION : COMPANY> pour voir sinon<|end_of_text|>


# saving

In [55]:
from huggingface_hub import notebook_login

notebook_login()

In [56]:
SAVE_NAME = "Llama-3.1-8B"

model.push_to_hub(f"benjaminzwhite/{SAVE_NAME}_pii-tags-test_LoRA") 
tokenizer.push_to_hub(f"benjaminzwhite/{SAVE_NAME}_pii-tags-test_LoRA")

README.md:   0%|          | 0.00/595 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/benjaminzwhite/Llama-3.1-8B_pii-tags-test_LoRA


In [ ]:
#quant_methods = ["q2_k", "q3_k_m", "q4_k_m", "q5_k_m", "q6_k", "q8_0"]
quant_methods = ["q4_k_m"]
for quant in quant_methods:
    model.push_to_hub_gguf(f"benjaminzwhite/{SAVE_NAME}_pii-tags-test_GGUF", tokenizer, quant)

Cloning into 'llama.cpp'...
Submodule 'kompute' (https://github.com/nomic-ai/kompute.git) registered for path 'ggml/src/kompute'
Cloning into '/kaggle/working/llama.cpp/ggml/src/kompute'...
Submodule path 'ggml/src/kompute': checked out '4565194ed7c32d1d2efa32ceab4d3c6cae006306'
make: Entering directory '/kaggle/working/llama.cpp'
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE -DGGML_USE_AMX  -std=c11   -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-un